단계별 구성
1. S3 버킷을 통한 문제 데이터 업로드
S3 버킷을 생성하고, 새로운 문제 데이터 파일이 업로드될 때마다 이를 감지할 수 있도록 S3 이벤트 알림을 설정합니다.
S3 이벤트 알림은 특정 조건(예: 업로드된 파일 수가 일정 수 이상일 때)을 만족할 경우 Lambda 함수가 트리거되도록 설정합니다.
2. Lambda 트리거 (InitialItemParameterSetup)
Lambda 함수 InitialItemParameterSetup은 새로운 문제 파일이 누적되었을 때 자동으로 트리거됩니다.
Lambda 함수는 다음 작업을 수행합니다:
S3에서 새로운 문제 데이터 파일을 확인합니다.
문제가 충분히 누적된 경우, EC2 인스턴스를 시작하여 베이지안 사전 추론 프로세스를 수행하도록 요청합니다.
EC2 인스턴스가 실행될 때 S3의 문제 데이터를 접근할 수 있도록 S3 접근 권한을 포함한 IAM 역할을 설정합니다.
3. EC2 인스턴스에서의 베이지안 사전 추론 작업
Lambda가 트리거한 EC2 인스턴스는 사전 설정된 AMI(Amazon Machine Image)를 사용하여 문제 데이터를 처리하도록 합니다.
EC2 인스턴스에 필요한 구성 요소는 다음과 같습니다:
S3에서 문제 데이터를 다운로드할 수 있는 스크립트.
베이지안 추론 코드 (Python 등으로 작성)와 필요한 패키지 설치 (boto3, pandas, scipy 등).
MariaDB 데이터베이스와 연결을 위한 데이터베이스 드라이버 및 설정.
EC2 내부의 구체적인 작업 흐름
문제 데이터 로드 및 전처리

S3에서 새로운 문제 데이터를 다운로드합니다.
문제 데이터에 필요한 전처리 작업을 수행합니다 (예: 데이터 정제, 결측치 처리).
베이지안 추론 실행

사전 설정된 분포를 기반으로 사전 분포를 설정합니다.
문제 데이터에 대해 우도 함수를 설정하고 관찰된 데이터를 바탕으로 사후 분포를 업데이트합니다.
사후 분포에서 최대 사후 확률(MAP) 추정을 통해 초기 문항 모수를 결정합니다.
문항 모수 저장

추정된 초기 문항 모수를 MariaDB에 저장합니다.
MariaDB는 EC2 내부에 설치할 수도 있고, RDS로 MariaDB를 생성해 별도로 접근할 수도 있습니다.
4. 문항 모수 저장 (MariaDB)
MariaDB를 활용하여 문항 모수를 저장합니다. EC2에서 추정된 모수 값을 다음과 같이 저장합니다:
문항 ID (문제 고유 식별자)
난이도, 변별도, 추측도 등 문항 모수 값
추가적으로, 추론 시 사용된 사전 정보 및 설정값을 기록해 두면 이후 분석에 유용할 수 있습니다.
5. 추가 설정 및 자동화 (선택 사항)
CloudWatch 알림: Lambda 함수 또는 EC2 인스턴스 작업 상태를 모니터링할 수 있도록 CloudWatch 알림을 설정해 실패나 오류가 발생할 경우 알림을 받을 수 있게 합니다.
종료 및 비용 관리: 작업이 완료되면 EC2 인스턴스가 자동 종료되도록 스크립트에 종료 명령을 추가하여 비용을 절약합니다.


1. Lambda에서 EC2 인스턴스 시작

In [ ]:
import boto3

def lambda_handler(event, context):
    ec2_client = boto3.client('ec2')
    # EC2 인스턴스 시작
    response = ec2_client.run_instances(
        ImageId='ami-xxxxxx',  # 미리 설정한 베이지안 추론용 AMI
        InstanceType='t2.medium',
        MinCount=1,
        MaxCount=1,
        KeyName='your-key-pair-name',
        IamInstanceProfile={'Name': 'EC2S3AccessRole'},  # S3 접근 가능한 역할
        UserData='''#!/bin/bash
            python3 /home/ec2-user/bayesian_inference_script.py
        '''
    )
    return response


2. EC2에서 S3 데이터 다운로드 및 베이지안 추론 수행 (Python 스크립트 예시)

In [ ]:
import boto3
import pandas as pd
from scipy.stats import norm
import mysql.connector

# S3에서 데이터 다운로드
s3 = boto3.client('s3')
s3.download_file('your-bucket-name', 'path/to/problem_data.csv', '/tmp/problem_data.csv')

# 데이터 로드 및 전처리
data = pd.read_csv('/tmp/problem_data.csv')

# 베이지안 사전 분포 설정 및 추론 실행
prior_difficulty = norm(loc=0, scale=1)  # 예시 사전 분포
likelihood_data = calculate_likelihood(data)  # 사용자 정의 함수

posterior = update_posterior(prior_difficulty, likelihood_data)  # 사후 분포 업데이트

# MariaDB에 저장
db = mysql.connector.connect(
    host="your-db-host",
    user="your-username",
    password="your-password",
    database="your-database"
)
cursor = db.cursor()

for item_id, params in posterior.items():
    query = "INSERT INTO item_parameters (item_id, difficulty, discrimination) VALUES (%s, %s, %s)"
    values = (item_id, params['difficulty'], params['discrimination'])
    cursor.execute(query, values)

db.commit()
cursor.close()
db.close()
